In [1]:
import openai

openai.api_key = 'aib_machinetranslation_a6a051'
openai.base_url = "https://pre-openai-keys.alibaba-inc.com"
model_name= "gpt-4o"

In [2]:
completion = openai.chat.completions.create(

    model=model_name,

    messages=[

    {
    "role": "user",

    "content": "请将以下文本翻译成中文：\n\n" + "Hello, how are you? I hope you are having a great day!"

    },

    ],

    max_tokens=100,

    temperature=0.7,

    top_p=0.9,

    frequency_penalty=0.0,

     presence_penalty=0.0,

    n=1
)

APITimeoutError: Request timed out.